<a href="https://colab.research.google.com/github/Amulya-B28/FutureIntern_AI_01/blob/main/txtgpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [ ]:
!pip install transformers datasets torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [1]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Loading Dataset

In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# Add a padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('gpt2')

from datasets import load_dataset

# Load your custom dataset (or a dataset from the Hugging Face hub)
dataset = load_dataset('Chadgpt-fam/sexting_dataset')

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

# Tokenize the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Training Dataset

In [3]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset

# Load pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# Add a padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Load your custom dataset (or a dataset from the Hugging Face hub)
dataset = load_dataset('Therax/ADHD_data')

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

# Tokenize the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    num_train_epochs=1,              # Number of training epochs
    per_device_train_batch_size=2,   # Batch size per device during training
    per_device_eval_batch_size=2,    # Batch size for evaluation
    warmup_steps=10,                # Warmup steps for learning rate scheduler
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
)

# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    # The dataset likely does not contain a validation split. Remove this or
    # split the training set if you require a validation set.
    # eval_dataset=tokenized_datasets['validation'],
)

# Fine-tune the model
# Set the labels to the input_ids
#tokenized_datasets = tokenized_datasets.with_format("torch")
trainer.train_dataset = tokenized_datasets["train"].add_column("labels", tokenized_datasets["train"]["input_ids"])
trainer.train(ignore_keys_for_eval=['past_key_values', 'hidden_states', 'attentions'])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=15, training_loss=4.536001586914063, metrics={'train_runtime': 796.1488, 'train_samples_per_second': 0.038, 'train_steps_per_second': 0.019, 'total_flos': 15677521920000.0, 'train_loss': 4.536001586914063, 'epoch': 1.0})

# Evaluating Dataset

In [4]:
split_datasets = tokenized_datasets["train"].train_test_split(test_size=0.2)
results = trainer.evaluate(eval_dataset=split_datasets['test'])
print(results)

{'eval_runtime': 36.9932, 'eval_samples_per_second': 0.162, 'eval_steps_per_second': 0.081, 'epoch': 1.0}


#Random prompt

In [5]:
prompt = "Once upon a time"
inputs = tokenizer.encode(prompt, return_tensors='pt')
outputs = model.generate(inputs, max_length=100, num_return_sequences=1)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Once upon a time, the world will be a better place.


In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

torch_device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
model = AutoModelForCausalLM.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id).to(torch_device)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


#Loading GPT2 Model and Tokenizing Dataset

In [7]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# Load GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Set the device (CPU or GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Tokenize the input from the dataset
input_text = dataset['train'][0]['text']  # Sample from the dataset
model_inputs = tokenizer(input_text, return_tensors='pt').to(device)


#Greedy Search

In [8]:
# Generate new tokens based on the dataset input
greedy_output = model.generate(**model_inputs, max_new_tokens=40)

# Decode the generated tokens into human-readable text
generated_text = tokenizer.decode(greedy_output[0], skip_special_tokens=True)

# Print the output
print("Generated Text:\n" + 100 * '-')
print(generated_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text:
----------------------------------------------------------------------------------------------------
How often have you been feeling excessive worry or nervousness, even about everyday things?

I've been feeling this way for a long time. I've been feeling this way for a long time. I've been feeling this way for a long time. I've been feeling this


#Beam search

In [9]:
# Activate beam search and early stopping
beam_output = model.generate(
    **model_inputs,
    max_new_tokens=40,     # Generate 40 new tokens
    num_beams=5,           # Use 5 beams for beam search
    early_stopping=True    # Stop early when the model finds a good output
)

# Decode the generated tokens into human-readable text
generated_text = tokenizer.decode(beam_output[0], skip_special_tokens=True)

# Print the output
print("Generated Text:\n" + 100 * '-')
print(generated_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text:
----------------------------------------------------------------------------------------------------
How often have you been feeling excessive worry or nervousness, even about everyday things?

I've been feeling this way for a long time. I've been feeling this way for a long time. I've been feeling this way for a long time. I've been feeling this


#Beam search with no repeated n-grams

In [10]:
# Activate
beam_output = model.generate(
    **model_inputs,
    max_new_tokens=40,          # Generate 40 new tokens
    num_beams=5,                # Use 5 beams for beam search
    no_repeat_ngram_size=2,     # Prevent repetition of 2-grams (two-word phrases)
    early_stopping=True         # Stop early when the model finds a good output
)

# Decode the generated tokens into human-readable text
generated_text = tokenizer.decode(beam_output[0], skip_special_tokens=True)

# Print the output
print("Generated Text:\n" + 100 * '-')
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text:
----------------------------------------------------------------------------------------------------
How often have you been feeling excessive worry or nervousness, even about everyday things?

I've had a lot of anxiety and panic attacks in the past few years, but I've never felt that way about anything. It's just that I don't know how to deal with


#Generate 5 sequences using beam search with no repeated n-grams

In [12]:
beam_outputs = model.generate(
    **model_inputs,
    max_new_tokens=60,          # Generate 40 new tokens
    num_beams=5,                # Use 5 beams for beam search
    no_repeat_ngram_size=2,     # Prevent repetition of 2-grams
    num_return_sequences=5,     # Generate and return 5 sequences
    early_stopping=True         # Stop early when a good output is found
)

# Now we have 5 output sequences
print("Generated Text:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
    print(f"Output {i + 1}:")
    print(tokenizer.decode(beam_output, skip_special_tokens=True))
    print(100 * '-')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text:
----------------------------------------------------------------------------------------------------
Output 1:
How often have you been feeling excessive worry or nervousness, even about everyday things?

I've had a lot of anxiety and panic attacks in the past few years, but I've never felt that way about anything. It's just that I don't know how to deal with it. I think it's because I'm so focused on my job and my family, and I
----------------------------------------------------------------------------------------------------
Output 2:
How often have you been feeling excessive worry or nervousness, even about everyday things?

I've had a lot of anxiety and panic attacks in the past few years, but I've never felt that way about anything. It's just that I don't know how to deal with it. I'm not sure if it's because of the stress, or if I just have a
----------------------------------------------------------------------------------------------------
Output 3:
How often ha

# Activate Sampling

In [13]:
sample_output = model.generate(
    **model_inputs,
    max_new_tokens=40,     # Generate 40 new tokens
    do_sample=True,        # Enable sampling for more diverse outputs
    top_k=0                # Allow sampling from the full distribution (no top-k restriction)
)

# Decode and print the generated text
print("Generated Text:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text:
----------------------------------------------------------------------------------------------------
How often have you been feeling excessive worry or nervousness, even about everyday things? What's the barbell-chest feeling like at night... Or at home, or apart from the workout or office routines that have inspired your TV habits?

How often have you discovered that adding


# Use temperature to decrease the sensitivity to low probability candidates


In [14]:
sample_output = model.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_k=0,
    temperature=0.6,
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
How often have you been feeling excessive worry or nervousness, even about everyday things? In the U.S., about 30% of Americans report having a reaction to something as "disordered." According to a recent survey of 5,000 U.S. adults, about half of


#Top-K Sampling

In [15]:
# set top_k to 50
sample_output = model.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_k=50
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
How often have you been feeling excessive worry or nervousness, even about everyday things?

It is usually one of those things. If you are stressed in your daily life and nervous, that is not healthy. And you are in extreme risk for panic attacks because your body is not


#Top-p (nucleus) sampling

In [16]:
# set top_k to 50
sample_output = model.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_p=0.92,
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
How often have you been feeling excessive worry or nervousness, even about everyday things?

Oh, and even when it feels like there's no point playing basketball, you always give me a break. I need other people to bring me new things. To bring me new games.


#Top P-Sampling

In [17]:
# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
sample_outputs = model.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    num_return_sequences=3,
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
0: How often have you been feeling excessive worry or nervousness, even about everyday things? It is because I try to make myself seem confident, healthy, and fit. Unfortunately, I have experienced these things quite often. I had to deal with an almost unbearable and scary feeling. I also
1: How often have you been feeling excessive worry or nervousness, even about everyday things?

You've probably experienced these. Maybe you're just a little afraid of the stress and worry it's going to get to you.

Maybe you've made yourself more and more anxious,
2: How often have you been feeling excessive worry or nervousness, even about everyday things? That's the big question facing me now—why do you think I'm experiencing it all so quickly?

Why do you think it has happened so fast?

Why does it matter that
